# BBC News Classification Project

There exists a large amount of information being stored in the electronic format. With such data it has
become a necessity of such means that could interpret and analyse such data and extract such facts that could
help in decision-making.<br>
News is easily accessible via content providers such as online news services. A huge amount of information exists in form of text in various diverse areas whose analysis can be beneficial in
several areas. Classification is quite a challenging field in text mining as it requires prepossessing steps to
convert unstructured data to structured information. With the increase in the number of news it has got difficult
for users to access news of his interest which makes it a necessity to categories news so that it could be easily
accessed. Categorization refers to grouping that allows easier navigation among articles. Internet news needs to
be divided into categories

### The Dataset
Text documents are one of the richest sources of data for businesses.

We’ll use a public dataset from the BBC comprised of 2225 articles, each labeled under one of 5 categories: business, entertainment, politics, sport or tech.

The dataset is broken into 1490 records for training and 735 for testing. The goal will be to build a system that can accurately classify previously unseen news articles into the right category.


1. **Summary of the project**: We are classifying BBC news articles into five categories using natural Language Processing and Machine Learning. The five news topics are: Politics, Entertainment, Sports, Technology, and Business. The goal of this project is to create a text classifier that will streamline the process of categorizing news publications. 
<br><br>
2. **Summary of the data**: The dataset consists of 2225 news articles extracted from the BBC website between 2004 and 2005. It was published open source by Insight Resources and was collected by UC Davis for research. The class distribution is as follows:
```
business: 510
entertainment: 386
politics: 417 
sports: 511
technology: 401 
```